# Rolling Planning mit Summe-Tardiness

In [1]:
import json
import pandas as pd
pd.set_option('display.max_rows', 40)

# Basics
import utils.basics.presenter as show
import utils.basics.converter as convert

import utils.checker as check
from ProductionDaySimulation import ProductionDaySimulation

In [2]:
import utils.rolling_planning.init_jobs_times as rp_init
import utils.rolling_planning.procedure as rp_proced

In [3]:
import utils.schedule_solver__arrival as ssv_a
import utils.reschedule.schedule_solver__arrival as rssv_a

In [4]:
import utils.schedule_solver__tardiness_plus as ssv_t
import utils.reschedule.schedule_solver__tardiness_plus as rssv_t

In [5]:
import time

## Generierung des Datensatzes für 6 Tage (Ankunft)

In [7]:
# Datei laden
with open("data/jobshop_instances.json", "r", encoding="utf-8") as f:
    jobshop_instances = json.load(f)

instance =  jobshop_instances["instance ft10"]
df_instance = convert.jssp_dict_to_df(instance)

df_jssp, df_times = rp_init.init_jobs_with_arrivals(df_instance, 8,  u_b_mmax = 0.90, generate_deadlines=True, deadlines_buffer_factor=1.20)
df_jssp

,Job,Operation,Machine,Processing Time
0,Job_000,0,M0,29
1,Job_000,1,M1,78
2,Job_000,2,M2,9
3,Job_000,3,M3,36
4,Job_000,4,M4,49
...,...,...,...,...
1515,Job_151,5,M9,76
1516,Job_151,6,M5,47
1517,Job_151,7,M3,52
1518,Job_151,8,M4,90


In [9]:
df_jssp.to_csv("data/06b_init_jssp.csv", index = False)
df_times.to_csv("data/06b_init_times.csv", index = False)

In [10]:
print(f"Mean Processing Time per Job:    {df_jssp.groupby('Job')['Processing Time'].sum().mean():.2f}")
print(f"Minimum Processing Time per Job: {df_jssp.groupby('Job')['Processing Time'].sum().min():.2f}")
print(f"Maximum Processing Time per Job: {df_jssp.groupby('Job')['Processing Time'].sum().max():.2f}")

Mean Processing Time per Job:    515.89
Minimum Processing Time per Job: 393.00
Maximum Processing Time per Job: 655.00


In [13]:
df_times

,Job,Arrival,Deadline
0,Job_000,0.00,1069.346415
1,Job_001,88.05,1468.725117
2,Job_002,152.78,1690.473072
3,Job_003,207.97,1981.190004
4,Job_004,246.61,1310.542002
...,...,...,...
147,Job_147,11270.89,13044.110004
148,Job_148,11277.28,12341.212002
149,Job_149,11278.72,12816.413072
150,Job_150,11404.04,12473.386415


In [12]:
solver_limit = 60*60*3 # 3h
solver_limit

1800

In [13]:
def get_schedule_filename(day: int, suffix: str = "", prefix: str = "06b") -> str:
    file_template = "data/{prefix}_schedule_{day:02d}{suffix}.csv"
    if suffix:
        suffix = f"_{suffix}"
    return file_template.format(prefix=prefix,day=day, suffix=suffix)


## Params

In [15]:
# fix
day_length = 1440
horizon_days = 3

file_template = "data/06_schedule_{day:02d}.csv"

In [16]:
day_numb = 0
planning_end = 0

# Initialisierung (Tag 0)

In [18]:
day_numb = 0

In [19]:
day_start = day_length*day_numb                      # 0
day_end = day_start + day_length                     # 1440
planning_end =  day_start + horizon_days*day_length  # 4320

In [20]:
# I)
df_jssp_curr, df_times_curr = rp_proced.filter_jobs_by_arrival_window(df_times, df_jssp, day_start, planning_end)
df_jssp_curr

,Job,Operation,Machine,Processing Time
0,Job_000,0,M0,29
1,Job_000,1,M1,78
2,Job_000,2,M2,9
3,Job_000,3,M3,36
4,Job_000,4,M4,49
...,...,...,...,...
565,Job_056,5,M9,76
566,Job_056,6,M5,47
567,Job_056,7,M3,52
568,Job_056,8,M4,90


In [21]:
df_times_curr 

,Job,Arrival,Deadline
0,Job_000,0.00,1514.907421
1,Job_001,88.05,2044.006416
2,Job_002,152.78,2331.178519
3,Job_003,207.97,2720.031672
4,Job_004,246.61,1753.847003
...,...,...,...
52,Job_052,4070.89,6582.951672
53,Job_053,4077.28,5584.517003
54,Job_054,4078.72,6257.118519
55,Job_055,4204.04,5718.947421


### Jobs vorziehen

In [23]:
# Test
df_jssp_beforehand, df_times_beforehand = rp_init.sample_jobs_with_times_after_arrivaltime(df_jssp, df_times, 
                                                                             min_arrival_time=planning_end,
                                                                             n=2, random_state=123)
df_times_beforehand

,Job,Arrival,Deadline
128,Job_128,9830.89,12342.951672
119,Job_119,8959.39,10861.653495


In [24]:
df_jssp_curr, df_times_curr = rp_init.add_beforehand_jobs_to_current_horizon(
    df_existing_jobs = df_jssp_curr,
    df_existing_times = df_times_curr,
    df_jssp = df_jssp,
    df_times = df_times,
    min_arrival_time = planning_end,
    n=3,
    random_state=23
)
df_times_curr

,Job,Arrival,Deadline
0,Job_000,0.00,1514.907421
1,Job_001,88.05,2044.006416
2,Job_002,152.78,2331.178519
3,Job_003,207.97,2720.031672
4,Job_004,246.61,1753.847003
...,...,...,...
55,Job_055,4204.04,5718.947421
56,Job_056,4309.61,6380.622676
57,Job_086,6651.94,8719.117468
58,Job_150,11404.04,12918.947421


### Scheduling

In [ ]:
info_time =  time.time()
df_plan = ssv_t.solve_jssp_sum_tardiness(df_jssp_curr, df_times_curr , solver_time_limit=solver_limit, threads = 8)
print(f"Dauer {(time.time() - info_time):.4f} Sekunden.")
df_plan

In [ ]:
df_plan.to_csv(get_schedule_filename(day=day_numb), index = False)

In [ ]:
show.plot_gantt_machines(df_plan)
check.check_all_constraints(df_plan)

In [ ]:
show.plot_gantt_jobs(df_plan)

### Simulation

In [ ]:
simulation = ProductionDaySimulation(df_plan, vc=0.35)
df_execution, df_undone = simulation.run(start_time = day_start, end_time=day_end)

In [ ]:
if not df_execution.empty:
    show.plot_gantt_machines(df_execution, duration_column="Simulated Processing Time")
else:
    print(f"Nothing executed on day {day_numb}")

In [ ]:
df_execution

In [ ]:
df_undone

In [ ]:
-

## Day 1

In [ ]:
day_numb = 1

day_start = day_length*day_numb                      # 1440
day_end = day_start + day_length                     # 2880
planning_end =  day_start + horizon_days*day_length  # 5760

In [ ]:
# I)
df_jssp_curr, df_times_curr = rp_proced.filter_jobs_by_arrival_window(df_times, df_jssp, day_start, planning_end)
df_jssp_curr

In [ ]:
# evtl. nicht gestarte Operationen ergänzen
df_jssp_curr = rp_proced.extend_with_undone_operations(df_jssp_curr, df_undone)
df_jssp_curr

In [ ]:
df_times_curr = rp_proced.update_times_after_operation_changes(df_times, df_jssp_curr)
df_times_curr

In [ ]:
# zusätzliche Jobs (vorzeitig)
df_jssp_curr, df_times_curr = rp_init.add_beforehand_jobs_to_current_horizon(
    df_existing_jobs = df_jssp_curr,
    df_existing_times = df_times_curr,
    df_jssp = df_jssp,
    df_times = df_times,
    min_arrival_time = planning_end,
    n=3,
    random_state=23
)
df_times_curr

In [ ]:
# relevante Bedingungen aus der Simulation
df_execution_important = rp_proced.get_operations_running_into_day(df_execution, day_start)
df_execution_important

### Rescheduling

#### a) r = 0.8

In [ ]:
this_r = 0.8 # 80 % Effizienz, 20 % Stabilität

In [ ]:
"""
df_reschedule = rssv_a.solve_jssp_bi_criteria_sum_tardiness_deviation_with_fixed_ops(
    df_jssp = df_jssp_curr,
    df_arrivals = df_times_curr,
    df_executed=df_execution_important,
    df_original_plan=df_plan,
    r = this_r, 
    solver_time_limit = solver_limit,
    reschedule_start = day_start,
    threads = 8
)
df_reschedule.to_csv(get_schedule_filename(day=day_numb, suffix=f"r{this_r:.3f}"), index = False)
"""

#### r = 0.4

In [ ]:
this_r = 0.4 # 40 % Effizienz, 60 % Stabilität

df_plan = rssv_a.solve_jssp_bi_criteria_sum_tardiness_deviation_with_fixed_ops(
    df_jssp = df_jssp_curr,
    df_arrivals = df_times_curr,
    df_executed=df_execution_important,
    df_original_plan=df_plan,
    r = this_r, 
    solver_time_limit = solver_limit,
    reschedule_start = day_start,
    threads = 8
)
df_plan

In [ ]:
show.plot_gantt_machines(df_plan, title = f"Gantt-Diagramm ab Tag {day_numb}")
check.check_all_constraints(df_plan)

In [ ]:
df_reschedule.to_csv(get_schedule_filename(day=day_numb), index = False)

In [ ]:
df_plan = df_reschedule

### Simulation

In [ ]:
simulation = ProductionDaySimulation(df_plan, vc=0.35)
df_execution, df_undone = simulation.run(start_time = day_start, end_time=day_end)

In [ ]:
if not df_execution.empty:
    show.plot_gantt_machines(df_execution, duration_column="Simulated Processing Time")
else:
    print(f"Nothing executed on day {day_numb}")

In [ ]:
df_undone

In [ ]:
-

## Day 2-5

In [ ]:
last_planning_start = 5

for day_numb in range(2, last_planning_start + 1):  # Schleife von 2 bis 5
    day_start = day_length*day_numb                     
    day_end = day_start + day_length                 
    planning_end =  day_start + horizon_days*day_length
    
    # Ia aktuelles Ankunftszeitfenster
    df_jssp_curr, df_times_curr = rp_proced.filter_jobs_by_arrival_window(df_times, df_jssp, day_start, planning_end)

    # Ib Operation ergänzen, die zuvor bekannt und nicht abgeschlossen (aber evtl. nicht aktuellen Zeitfenster) 
    df_jssp_curr = rp_proced.extend_with_undone_operations(df_jssp_curr, df_undone)

    # Ic Zeiten-Dataframe aktualisieren (wegen Ib)
    df_times_curr = rp_proced.update_times_after_operation_changes(df_times, df_jssp_curr)


    # II drei zusätzliche Jobs (vorzeitig)
    df_jssp_curr, df_times_curr = rp_init.add_beforehand_jobs_to_current_horizon(
        df_existing_jobs = df_jssp_curr,
        df_existing_times = df_times_curr,
        df_jssp = df_jssp,
        df_times = df_times,
        min_arrival_time = planning_end,
        n=3,
        random_state=23
    )

    # III relevante Bedingungen aus der Simulation (Operationen, die in aktuellen Planunghorizont hineinlaufen)
    df_execution_important = rp_proced.get_operations_running_into_day(df_execution, day_start)

    # Rescheduling
    this_r = 0.4   # 40 % Effizienz, 60 % Stabilität

    df_plan = rssv_a.solve_jssp_bi_criteria_flowtime_deviation_with_fixed_ops(
        df_jssp = df_jssp_curr,
        df_arrivals = df_times_curr,
        df_executed=df_execution_important,
        df_original_plan=df_plan,
        r = this_r, 
        solver_time_limit = solver_limit,
        reschedule_start = day_start,
        threads = 8
    )
    df_plan.to_csv(get_schedule_filename(day=day_numb), index = False)

    show.plot_gantt_machines(df_plan, title = f"Gantt-Diagramm ab Tag {day_numb}")
    check.check_all_constraints(df_plan)

    # Simulation (ein Tag)
    simulation = ProductionDaySimulation(df_plan, vc=0.35)
    df_execution, df_undone = simulation.run(start_time = day_start, end_time=day_end)
    show.plot_gantt_machines(df_execution, 
                             title = f"Gantt-Diagramm für Simulationstag {day_numb}",
                             duration_column = "Simulated Processing Time")